In [168]:
import pandas as pd
import geopandas as gpd
from rasterstats import zonal_stats
import rasterio as rio

In [163]:
path_to_cell_data_section = "C:/Users/Luc/projects/goodwin_creek/TOPAGNPS/topagnps_goodwin_creek_3m_csa_15_mscl_100/run2/AnnAGNPS_Cell_Data_Section.csv"
path_to_soil_shp = "C:/Users/Luc/projects/goodwin_creek/datasets/soil_data/ssurgo_soil_layer.shp"


path_to_cells_shp = "C:/Users/Luc/projects/goodwin_creek/TOPAGNPS/topagnps_goodwin_creek_3m_csa_15_mscl_100_onerun/AnnAGNPS_Cell_IDs.shp"

In [165]:
cells_csv = pd.read_csv(path_to_cell_data_section)
cells_csv

,Cell_ID,Soil_ID,Mgmt_Field_ID,Reach_ID,Reach_Location_Code,Cell_Area,Time_of_Conc,Avg_Elevation,RCN_Calib_ID,Avg_Land_Slope,...,Shallow_Conc_Flow_Length,Delivery_Ratio,Constant_USLE_C_Fctr,Constant_USLE_P_Fctr,All_OC_Calib_Fctr,All_N_Calib_Fctr,All_P_Calib_Fctr,Sheet_and_Rill_Erosion_Calib_Fctr,Gullies_Erosion_Calib_Fctr,Input_Units_Code
0,22,NaN,NaN,2,1,4.7979,NaN,71.75,NaN,0.06721,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,23,NaN,NaN,2,1,25.6113,NaN,78.86,NaN,0.11792,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,31,NaN,NaN,3,0,17.5698,NaN,82.33,NaN,0.08892,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,32,NaN,NaN,3,1,1.0053,NaN,73.08,NaN,0.09088,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,33,NaN,NaN,3,1,8.0532,NaN,78.22,NaN,0.06188,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,732,NaN,NaN,73,1,8.0955,NaN,79.10,NaN,0.05894,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
179,733,NaN,NaN,73,1,5.5044,NaN,84.51,NaN,0.11087,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
180,741,NaN,NaN,74,0,15.0075,NaN,97.41,NaN,0.10225,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
181,742,NaN,NaN,74,1,14.0517,NaN,85.58,NaN,0.10066,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [166]:
cells_shp = gpd.read_file(path_to_cells_shp)

In [159]:
soil_shp = gpd.read_file(path_to_soil_shp)
soil_shp[['musym', 'muname', 'geometry']]

,musym,muname,geometry
0,Gu,"Gullied land, silty","POLYGON ((233391.166 3792991.669, 233450.283 3..."
1,GrB2,"Grenada silt loam, 2 to 5 percent slopes, eroded","POLYGON ((235404.986 3794509.287, 235513.912 3..."
2,LoE2,"Loring silt loam, 12 to 17 percent slopes, eroded","POLYGON ((232489.321 3792777.497, 232500.055 3..."
3,Gu,"Gullied land, silty","POLYGON ((234145.603 3792430.322, 234145.603 3..."
4,GrC3,"Grenada silt loam, 5 to 8 percent slopes, seve...","POLYGON ((232327.923 3792523.349, 232352.499 3..."
...,...,...,...
307,Co,"Collins silt loam, local alluvium (occasionall...","POLYGON ((239911.530 3795230.325, 239913.538 3..."
308,Gu,"Gullied land, silty","POLYGON ((237999.521 3795023.216, 238000.560 3..."
309,LoE2,"Loring silt loam, 12 to 17 percent slopes, eroded","POLYGON ((236956.134 3796063.106, 237061.535 3..."
310,Gu,"Gullied land, silty","POLYGON ((239851.603 3795694.322, 239860.428 3..."


In [160]:
CRS = cells_shp.crs
for _, cell in cells_shp.iterrows():
    gdf_cell = gpd.GeoDataFrame(pd.DataFrame(data={'DN':[cell[0]], 
                                               'geometry':[cell[1]]}), 
                                             crs=CRS)
    intersection = soil_shp.overlay(gdf_cell, how='intersection', keep_geom_type=False)
    intersection['area'] = intersection.geometry.area
    intersection = pd.pivot_table(intersection, index=['musym'], values=['area'], aggfunc='sum') # !!! TEST THIS (to make sure if there are multiple intersections with the same soil group it's counted as one)
    musymmajority = intersection.loc[intersection['area'].idxmax(),'musym']
    cells_csv.loc[cells_csv['Cell_ID'] == cell[0],'Soil_ID'] = musymmajority

cells_csv
# cell


,Cell_ID,Soil_ID,Mgmt_Field_ID,Reach_ID,Reach_Location_Code,Cell_Area,Time_of_Conc,Avg_Elevation,RCN_Calib_ID,Avg_Land_Slope,...,Shallow_Conc_Flow_Length,Delivery_Ratio,Constant_USLE_C_Fctr,Constant_USLE_P_Fctr,All_OC_Calib_Fctr,All_N_Calib_Fctr,All_P_Calib_Fctr,Sheet_and_Rill_Erosion_Calib_Fctr,Gullies_Erosion_Calib_Fctr,Input_Units_Code
0,22,Fa,NaN,2,1,4.7979,NaN,71.75,NaN,0.06721,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,23,Gu,NaN,2,1,25.6113,NaN,78.86,NaN,0.11792,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,31,Fl,NaN,3,0,17.5698,NaN,82.33,NaN,0.08892,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,32,Fa,NaN,3,1,1.0053,NaN,73.08,NaN,0.09088,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,33,Fa,NaN,3,1,8.0532,NaN,78.22,NaN,0.06188,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,732,LoE2,NaN,73,1,8.0955,NaN,79.10,NaN,0.05894,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
179,733,LoE2,NaN,73,1,5.5044,NaN,84.51,NaN,0.11087,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
180,741,MnF2,NaN,74,0,15.0075,NaN,97.41,NaN,0.10225,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
181,742,MnF2,NaN,74,1,14.0517,NaN,85.58,NaN,0.10066,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [161]:
# cells_csv.to_csv('panola.csv',index=False)

In [211]:
nlcd_values = {11: 'Open Water',
               12: 'Perennial Ice/Snow',
               21: 'Developed, Open Space',
               22: 'Developed, Low Intensity',
               23: 'Developed, Medium Intensity',
               24: 'Developed High Intensity',
               31: 'Barren Land (Rock/Sand/Clay)',
               32: 'Unconsolidated Shore', # Not in LU keyfile
               41: 'Deciduous Forest', 
               42: 'Evergreen Forest',
               43: 'Mixed Forest',
               51: 'Dwarf Scrub (AK only)', # Not in LU keyfile
               52: 'Shrub/Scrub', # LU keyfile has two instances of Shrubland
               71: 'Grassland/Herbaceous', # Not in LU keyfile (has Grass/Pasture or Herbs, Herbaceous Wetlands)
               72: 'Sedge/Herbaceous (AK only)', # Not in LU keyfile
               73: 'Lichens (AK only)', # Not in LU keyfile
               74: 'Moss (AK only)', # Not in LU keyfile 
               81: 'Pasture/Hay', # Not in LU keyfile (has Grass/Pasture)
               82: 'Cultivated Crops', # Not in LU keyfile (has all sorts of Crops)
               90: 'Woody Wetlands',
               95: 'Emergent Herbaceous Wetlands' # Has Herbaceous Wetland
               }